## Load Annotations and Map IDs

In [2]:
import os
import json

# Paths to test set folder and annotation file
test_images_folder = r"C:\Users\shali\Documents\L&D\GitHub Projects\Machine Learning\003_LIVECell_Segmentation of Cells_U-Net\images\test set"
test_annotations_path = r"C:\Users\shali\Documents\L&D\GitHub Projects\Machine Learning\003_LIVECell_Segmentation of Cells_U-Net\annotations\LIVECell_test.json"

# Load the annotation file
with open(test_annotations_path, 'r') as f:
    annotations = json.load(f)

# Map image filenames to annotations
image_filename_to_id = {image['file_name']: image['id'] for image in annotations['images']}
id_to_annotations = {}
for annotation in annotations['annotations']:
    image_id = annotation['image_id']
    if image_id not in id_to_annotations:
        id_to_annotations[image_id] = []
    id_to_annotations[image_id].append(annotation)

print(f"Total images in test set: {len(os.listdir(test_images_folder))}")
print(f"Total images matched with annotations: {len(image_filename_to_id)}")

Total images in test set: 1512
Total images matched with annotations: 1512


## Preprocess Images and Generate Masks

In [4]:
import numpy as np
import cv2

# Function to preprocess test set
def preprocess_test_set(images_folder, img_size=(512, 512)):
    images = []
    masks = []

    for image_filename in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_filename)

        # Load and preprocess the image
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, img_size) / 255.0  # Resize and normalize
        images.append(image)

        # Generate binary mask
        mask = np.zeros(img_size, dtype=np.uint8)
        if image_filename in image_filename_to_id:
            image_id = image_filename_to_id[image_filename]
            if image_id in id_to_annotations:
                for ann in id_to_annotations[image_id]:
                    for seg in ann['segmentation']:
                        poly = np.array(seg).reshape((-1, 2))
                        scale_x = img_size[0] / annotations['images'][0]['width']
                        scale_y = img_size[1] / annotations['images'][0]['height']
                        poly[:, 0] *= scale_x
                        poly[:, 1] *= scale_y
                        poly = np.round(poly).astype(np.int32)
                        cv2.fillPoly(mask, [poly], 1)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Preprocess the test set
X_test, y_test = preprocess_test_set(test_images_folder)

# Check shapes
print(f"Test data shape: {X_test.shape}, Test masks shape: {y_test.shape}")

Test data shape: (1512, 512, 512, 3), Test masks shape: (1512, 512, 512)


In [6]:
# Ensure test masks have the correct shape
if len(y_test.shape) == 3:  # Check if channel dimension is missing
    y_test = np.expand_dims(y_test, axis=-1)  # Add channel dimension
    print(f"Updated Test masks shape: {y_test.shape}")

Updated Test masks shape: (1512, 512, 512, 1)


## Load the Model

In [8]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Redefine custom metrics
def iou_metric(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Threshold predictions at 0.5
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return intersection / (union + K.epsilon())  # Add epsilon to avoid division by zero

def dice_coefficient(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Threshold predictions at 0.5
    intersection = K.sum(y_true * y_pred)
    dice = (2. * intersection) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())
    return dice

# Load the saved model
from tensorflow.keras.models import load_model

model_path = r"C:\Users\shali\Documents\L&D\GitHub Projects\Machine Learning\003_LIVECell_Segmentation of Cells_U-Net\output_model\unet_trained_model.h5"
trained_model = load_model(
    model_path,
    custom_objects={"iou_metric": iou_metric, "dice_coefficient": dice_coefficient}
)
print("Model loaded successfully!")

Model loaded successfully!


### Data Preparation and Preprocessing prior to model evaluation

In [10]:
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_test shape: (1512, 512, 512, 3)
y_test shape: (1512, 512, 512, 1)


In [12]:
print(f"X_test dtype: {X_test.dtype}")
print(f"y_test dtype: {y_test.dtype}")

X_test dtype: float64
y_test dtype: uint8


In [14]:
# Convert X_test to float32
X_test = X_test.astype(np.float32)

# Convert y_test to float32 and ensure binary values (0 or 1)
y_test = y_test.astype(np.float32)
y_test = (y_test > 0.5).astype(np.float32)  # Optional: Ensure binary masks if not already

In [15]:
print(f"X_test dtype: {X_test.dtype}")
print(f"y_test dtype: {y_test.dtype}")
print(f"Unique values in y_test: {np.unique(y_test)}")

X_test dtype: float32
y_test dtype: float32
Unique values in y_test: [0. 1.]


## Quantitative Model Evaluation

In [18]:
test_loss, test_accuracy, test_iou, test_dice = trained_model.evaluate(X_test, y_test, verbose=1)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test IoU: {test_iou:.4f}")
print(f"Test Dice Coefficient: {test_dice:.4f}")

48/48 ━━━━━━━━━━━━━━━━━━━━ 351s 7s/step - accuracy: 0.8799 - dice_coefficient: 0.7790 - iou_metric: 0.6530 - loss: 0.2858
Test Loss: 0.3606
Test Accuracy: 0.8384
Test IoU: 0.6112
Test Dice Coefficient: 0.7493


# TO BE UPDATED FURTHER - WORK IN PROGRESS